In [1]:
import pandas as pd
from nba_api.stats.endpoints import commonallplayers, playergamelog
import time

def get_all_players(season='2024-25'):
    """Return dataframe of all players active in that season (with IDs and names)."""
    cap = commonallplayers.CommonAllPlayers(season=season, 
                                            league_id='00', 
                                            is_only_current_season=1)
    df = cap.get_data_frames()[0]
    # keep useful columns
    df = df[['PERSON_ID', 'DISPLAY_FIRST_LAST']]
    return df

def get_player_logs(player_id, season='2024-25'):
    """Get the game logs for that player in the given season."""
    try:
        gl = playergamelog.PlayerGameLog(player_id=player_id, season=season, season_type_all_star='Regular Season')
        df_gl = gl.get_data_frames()[0]
        return df_gl
    except Exception as e:
        print(f"Error fetching for player {player_id}: {e}")
        return pd.DataFrame()

def build_filtered_logs(season='2024-25', min_minutes=1):
    players_df = get_all_players(season)
    all_logs = []
    for _, row in players_df.iterrows():
        pid = row['PERSON_ID']
        logs = get_player_logs(pid, season)
        if logs.empty:
            continue

        # Convert 'MIN' to numeric minutes
        if logs['MIN'].dtype == 'object':
            def parse_min(s):
                if isinstance(s, str) and ':' in s:
                    parts = s.split(':')
                    return int(parts[0]) + int(parts[1]) / 60
                return pd.to_numeric(s, errors='coerce')
            logs['MIN_num'] = logs['MIN'].apply(parse_min)
        else:
            logs['MIN_num'] = logs['MIN']

        # Filter
        logs2 = logs[logs['MIN_num'] > min_minutes]

        # Attach player name
        logs2['PLAYER_NAME'] = row['DISPLAY_FIRST_LAST']

        all_logs.append(logs2)
        time.sleep(0.6)  # avoid rate limit

    full_df = pd.concat(all_logs, ignore_index=True)
    return full_df

# Usage
df_min15 = build_filtered_logs(season='2024-25', min_minutes=1)



C:\Users\zivaz\anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [2]:
print(df_min15)

      SEASON_ID  Player_ID     Game_ID     GAME_DATE      MATCHUP WL  MIN  \
0         22024    1630173  0022401188  Apr 13, 2025    NYK @ BKN  W   33   
1         22024    1630173  0022401175  Apr 11, 2025  NYK vs. CLE  L   15   
2         22024    1630173  0022401167  Apr 10, 2025    NYK @ DET  L   40   
3         22024    1630173  0022401128  Apr 05, 2025    NYK @ ATL  W   25   
4         22024    1630173  0022401103  Apr 02, 2025    NYK @ CLE  L   21   
...         ...        ...         ...           ...          ... ..  ...   
25810     22024     203967  0022400139  Nov 01, 2024    DEN @ MIN  L    5   
25811     22024     203967  0022400113  Oct 29, 2024    DEN @ BKN  W   12   
25812     22024     203967  0022400107  Oct 28, 2024    DEN @ TOR  W    5   
25813     22024     203967  0022400087  Oct 26, 2024  DEN vs. LAC  L   11   
25814     22024     203967  0022400075  Oct 24, 2024  DEN vs. OKC  L   11   

       FGM  FGA  FG_PCT  ...  AST  STL  BLK  TOV  PF  PTS  PLUS_MINUS  \
0 

In [3]:
print(df_min15.columns.tolist())

['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE', 'MIN_num', 'PLAYER_NAME']


In [4]:
df_min15.drop(['SEASON_ID','Game_ID','GAME_DATE','MATCHUP','WL','MIN','PF','PLUS_MINUS','VIDEO_AVAILABLE'], axis=1, inplace=True)

In [5]:
df_min15.to_csv('data.csv')